In [ ]:
#1 - Etapa de importações:
import pandas as pd
import numpy as np

: 

In [ ]:
#1.1 - Importação da database
df_imoveis_SP = pd.read_csv("./sao-paulo-properties-april-2019.csv")
df_imoveis_SP.head()

In [ ]:
#2 - Tratamento de dados:

'''Como a idéia dessa análise de dados é se basear em
aluguéis de imóveis em São Paulo, devemos fazer uma abstração
dos dados, tendo somente a disponibilidade de aluguéis'''

df_aluguel = df_imoveis_SP[df_imoveis_SP["Negotiation Type"] == "rent"]

#2.1 - Verificando a qualidade dos dados para tratamento, no caso, se existe valores nulos ou de tipagem incorreta
df_aluguel.info()


In [ ]:
#3 - Anállise Exploratória dos dados

#3.1 - Verificando a distribuição dos dados de acordo com cada variável
df_aluguel.hist(bins = 20, figsize=(15, 10))


In [ ]:
#3.2 - Análise de correlação dos dados com matriz de correlação
'''A análise de correlação, resumidamente, mede o grau de relação
linear entre duas variáveis numéricas, trazendo direção (correlação 
positiva e negativa) e intensidade'''

#3.2  Análise de correlação dos dados com matriz de correlação
'''Para utilizar esse método de observação, devemos escolher apenas
as colunas numéricas, descritas no tópico de verificação de qualidade
das informações do DF'''

colunas_numericas = df_aluguel.select_dtypes(include=['int64', 'float64']).columns
correlacao_price = df_aluguel[colunas_numericas].corr()["Price"].sort_values(ascending=False)
correlacao_price

In [ ]:
df_cleaned = df_aluguel.drop(["New","Property Type","Negotiation Type"], axis=1)
df_cleaned

In [ ]:
one_hot = pd.get_dummies(df_cleaned["District"])
df = df_cleaned.drop("District", axis = 1)
df = df.join(one_hot)
df

In [ ]:
from sklearn.model_selection import train_test_split
Y = df["Price"]
X = df.loc[:, df.columns != "Price"]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

In [ ]:
# Regressão Linear
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(x_train, y_train) # Aqui ele vai utilizar a função custo para fazer a otimização do modelo

In [ ]:
import pandas as pd

alguns_dados = x_train.iloc[:5]
alguns_label = y_train.iloc[:5]
predicoes = lin_reg.predict(alguns_dados).round(2)
labels = alguns_label.values

# Monta o DataFrame
df_result = pd.DataFrame({
    'Predições':       predicoes,
    'Valores reais':   labels
})
df_result

In [ ]:
import seaborn as sns
# Cria o gráfico
ax = sns.barplot(
    x=df_result.index, 
    y='Predições', 
    data=df_result,
    alpha=0.7,
    label='Predições'
)
sns.barplot(
    x=df_result.index, 
    y='Valores reais', 
    data=df_result,
    alpha=0.7,
    label='Valores reais',
    dodge=False,
    ax=ax
)

ax.set_xlabel('Índice')
ax.set_ylabel('Preço')
ax.set_title('Predições vs Valores Reais')
ax.legend()


In [ ]:
# Avaliação do modelo de dados

from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# predições no conjunto de treinamento
preds = lin_reg.predict(x_train)

# MSE e RMSE
lin_mse = mean_squared_error(y_train, preds)
lin_rmse = np.sqrt(lin_mse)

# R2
lin_r2 = r2_score(y_train, preds)

print(f"RMSE: {lin_rmse:.4f}")
print(f"R²:   {lin_r2:.4f}")